In [ ]:
import torch
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Path to the image
image_path = '/content/74530.png'
output_image_path = '/content/74530_with_boxes.png'  # Path to save the output image

# Initialize the YOLOv5 model (using a larger model 'yolov5x')
model = torch.hub.load('ultralytics/yolov5', 'yolov5x')

# Set lower confidence threshold and NMS IoU threshold
model.conf = 0.25  # Confidence threshold
model.iou = 0.45   # NMS IoU threshold

# Read the image
img = cv2.imread(image_path)

# Check if the image was loaded successfully
if img is not None:
    # Get original image dimensions
    orig_height, orig_width, _ = img.shape

    # Convert the image from BGR to RGB format
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Optionally resize the image to 640x640 for inference
    img_resized = cv2.resize(img_rgb, (640, 640))

    # Perform object detection
    results = model(img_resized)

    # Print the results for debugging
    print(results)  # Check the structure of results

    # Extract detected object names and bounding boxes
    detections = results.pandas().xyxy[0]
    if not detections.empty:
        object_names = detections['name'].unique()
        print("Detected objects:", object_names)

        # Scaling factor between resized image (640x640) and original image
        x_scale = orig_width / 640
        y_scale = orig_height / 640

        # Draw bounding boxes on the original image (RGB format)
        for i in range(len(detections)):
            x_min, y_min, x_max, y_max = detections.iloc[i][['xmin', 'ymin', 'xmax', 'ymax']]
            confidence = detections.iloc[i]['confidence']
            class_id = detections.iloc[i]['class']
            label = results.names[int(class_id)]

            # Scale bounding boxes to match original image size
            x_min = int(x_min * x_scale)
            y_min = int(y_min * y_scale)
            x_max = int(x_max * x_scale)
            y_max = int(y_max * y_scale)

            # Draw rectangle (bounding box) on the original image
            cv2.rectangle(img_rgb, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

            # Adjust the size of the label text
            font_scale = 0.5  # Adjust this to change the text size (0.5 is smaller, increase for larger)
            font_thickness = 1  # You can also adjust the thickness of the text

            # Get the size of the text
            text = f'{label} {confidence:.2f}'
            (text_width, text_height), baseline = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_thickness)

            # Draw a filled rectangle (red) as a background for the text
            cv2.rectangle(img_rgb, (x_min, y_min - text_height - 5),
                          (x_min + text_width, y_min), (0, 0, 255), -1)

            # Put label text (white color) on top of the red rectangle
            cv2.putText(img_rgb, text, (x_min, y_min - 5), cv2.FONT_HERSHEY_SIMPLEX,
                        font_scale, (255, 255, 255), font_thickness)

        # Save the original image with bounding boxes (converted back to BGR)
        img_bgr = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR)
        cv2.imwrite(output_image_path, img_bgr)
        print(f"Image saved to {output_image_path}")

        # Display the original image with bounding boxes (in RGB)
        plt.figure(figsize=(10, 10))
        plt.imshow(img_rgb)
        plt.axis('off')
        plt.show()
    else:
        print("No objects detected.")
else:
    print(f"Failed to load image: {image_path}")


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-9-27 Python-3.10.12 torch-2.4.1+cu121 CPU

Fusing layers... 
YOLOv5x summary: 444 layers, 86705005 parameters, 0 gradients, 205.5 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


image 1/1: 640x640 1 person, 1 tie, 2 forks, 1 knife, 1 bowl, 1 pizza, 4 chairs
Speed: 13.3ms pre-process, 4004.3ms inference, 1.4ms NMS per image at shape (1, 3, 640, 640)
Detected objects: ['knife' 'pizza' 'person' 'fork' 'tie' 'bowl' 'chair']
Image saved to /content/74530_with_boxes.png


In [6]:
import torch
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import os
import zipfile
from collections import Counter
from google.colab import drive


drive.mount('/content/drive')

# Path to the ZIP file on Google Drive
zip_file_path = '/content/drive/MyDrive/meme/image.zip'

# Path where the ZIP file will be extracted
extracted_folder = '/content/extracted_images/'

# Path to save the CSV file
output_csv_path = '/content/detected_objects.csv'

# Initialize the YOLOv5 model (using a larger model 'yolov5x')
model = torch.hub.load('ultralytics/yolov5', 'yolov5x')

# Set lower confidence threshold and NMS IoU threshold
model.conf = 0.25  # Confidence threshold
model.iou = 0.45   # NMS IoU threshold

# Function to extract the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder)

# List all image files in the extracted folder
# image_files = [os.path.join(extracted_folder, img) for img in os.listdir(extracted_folder) if img.endswith(('.png', '.jpg', '.jpeg'))]
image_files = []
for subdir, dirs, files in os.walk(extracted_folder):
  for file in files:
    if file.endswith(('.png', '.jpg', '.jpeg')):
      image_files.append(os.path.join(subdir, file))
# Create a dictionary to store object counts
object_counter = Counter()

# Create a list to store CSV data
csv_data = []

# Process each image in the extracted folder
for image_path in image_files:
    img_id = os.path.basename(image_path).split('.')[0]  # Extract image ID (filename without extension)

    # Read the image
    img = cv2.imread(image_path)

    # Check if the image was loaded successfully
    if img is not None:
        # Convert the image from BGR to RGB format
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Optionally resize the image to 640x640 for inference
        img_resized = cv2.resize(img_rgb, (640, 640))

        # Perform object detection
        results = model(img_resized)

        # Extract detected object names
        detections = results.pandas().xyxy[0]
        if not detections.empty:
            object_names = detections['name'].tolist()
            object_counter.update(object_names)  # Update the overall object counter
            detected_objects = ', '.join(object_names)

            # Append to CSV data (image ID and detected objects)
            csv_data.append([img_id, detected_objects])

# Write CSV data to a file
df = pd.DataFrame(csv_data, columns=['id', 'detected_objects'])
df.to_csv(output_csv_path, index=False)
print(f"Detected objects written to CSV file: {output_csv_path}")

# Plot a pie chart for object distribution
labels, counts = zip(*object_counter.items())
plt.figure(figsize=(8, 8))
plt.pie(counts, labels=labels, autopct='%1.1f%%', startangle=90)
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.title('Distribution of Detected Objects')
plt.show()


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-9-27 Python-3.10.12 torch-2.4.1+cu121 CPU



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Fusing layers... 
YOLOv5x summary: 444 layers, 86705005 parameters, 0 gradients, 205.5 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autoc

KeyboardInterrupt: 

In [1]:
import torch
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import os
import zipfile
from collections import Counter
from google.colab import drive


drive.mount('/content/drive')

# Path to the ZIP file on Google Drive
zip_file_path = '/content/drive/MyDrive/meme/image.zip'

# Path where the ZIP file will be extracted
extracted_folder = '/content/extracted_images/'

# Path to save the CSV file
output_csv_path = '/content/detected_objects.csv'

# Initialize the YOLOv5 model (using a larger model 'yolov5x')
model = torch.hub.load('ultralytics/yolov5', 'yolov5x')

# Set lower confidence threshold and NMS IoU threshold
model.conf = 0.25  # Confidence threshold
model.iou = 0.45   # NMS IoU threshold

# Function to extract the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder)

# List all image files in the extracted folder
image_files = []
for subdir, dirs, files in os.walk(extracted_folder):
  for file in files:
    if file.endswith(('.png', '.jpg', '.jpeg')):
      image_files.append(os.path.join(subdir, file))

# Create a dictionary to store object counts
object_counter = Counter()

# Create a list to store CSV data
csv_data = []

# Total number of images
total_images = len(image_files)

# Process each image in the extracted folder
for idx, image_path in enumerate(image_files, start=1):
    img_id = os.path.basename(image_path).split('.')[0]  # Extract image ID (filename without extension)

    # Read the image
    img = cv2.imread(image_path)

    # Check if the image was loaded successfully
    if img is not None:
        # Convert the image from BGR to RGB format
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Optionally resize the image to 640x640 for inference
        img_resized = cv2.resize(img_rgb, (640, 640))

        # Perform object detection
        results = model(img_resized)

        # Extract detected object names
        detections = results.pandas().xyxy[0]
        if not detections.empty:
            object_names = detections['name'].tolist()
            object_counter.update(object_names)  # Update the overall object counter
            detected_objects = ', '.join(object_names)

            # Append to CSV data (image ID and detected objects)
            csv_data.append([img_id, detected_objects])

    # Display progress
    print(f"Processed {idx}/{total_images} images.")

# Write CSV data to a file
df = pd.DataFrame(csv_data, columns=['id', 'detected_objects'])
df.to_csv(output_csv_path, index=False)
print(f"Detected objects written to CSV file: {output_csv_path}")

# Plot a pie chart for object distribution
labels, counts = zip(*object_counter.items())
plt.figure(figsize=(8, 8))
plt.pie(counts, labels=labels, autopct='%1.1f%%', startangle=90)
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.title('Distribution of Detected Objects')
plt.show()


Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/torch/hub.py:295: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
requirements: Ultralytics requirement ['gitpython>=3.1.30'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 245.1 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 3.0s, installed 1 package: ['gitpython>=3.1.30']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



YOLOv5 🚀 2024-9-27 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

100%|██████████| 166M/166M [00:01<00:00, 138MB/s]

Fusing layers... 
YOLOv5x summary: 444 layers, 86705005 parameters, 0 gradients, 205.5 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
WARNING ⚠️ NMS time limit 0.550s exceeded
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Processed 1/10000 images.
Processed 2/10000 images.
Processed 3/10000 images.
Processed 4/10000 images.
Processed 5/10000 images.


...

/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Processed 9995/10000 images.
Processed 9996/10000 images.
Processed 9997/10000 images.


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Processed 9998/10000 images.
Processed 9999/10000 images.
Processed 10000/10000 images.
Detected objects written to CSV file: /content/detected_objects.csv


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
